In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train=pd.read_csv("../input/train.csv")

In [ ]:
train.shape

In [ ]:
train.describe()

In [ ]:
train.isnull().sum()

Shows that the response variable is Right Skewed
So log transformation can be applied to decrese skewness

In [ ]:
plt.hist(train["SalePrice"],bins=50)

In [ ]:
plt.hist(np.log(train["SalePrice"]),bins=50)

In [ ]:
plt.scatter(train["YearBuilt"],train["SalePrice"])

In [ ]:
y1=np.log(train["SalePrice"])

There can be seen a linear type relation between Year Built and Sale Price

In [ ]:
plt.scatter(train["YearBuilt"],y1)

In [ ]:
numerical=list(train.describe().columns)

Attempt to see relation between all numerical variables and Sale Price

In [ ]:
plt.figure(figsize=(20,10))
for i in numerical:
    if (i!="SalePrice"):
        plt.scatter(train[i],train["SalePrice"],label=i)
        plt.legend(loc=4)

Sale Price has Many outliers

In [ ]:
e=plt.boxplot(train["SalePrice"])


In [ ]:
iqr=train["SalePrice"].quantile(0.75)-train["SalePrice"].quantile(0.25)
iqr1=train["SalePrice"].quantile(0.25)-1.5*iqr
iqr2=train["SalePrice"].quantile(0.75)+1.5*iqr

Removing Outliers and storing in new dataframe

In [ ]:
rem_outlier=(train["SalePrice"] <= iqr2) & (train["SalePrice"] >= iqr1)

In [ ]:
train2=train[rem_outlier]

In [ ]:
y=np.log(train2["SalePrice"])

In [ ]:
plt.scatter(train2["YearBuilt"],y)

Inserting A linear model 

In [ ]:
p=np.polyfit(train2["YearBuilt"],train2["SalePrice"],1)


c=np.poly1d(p)

plt.scatter(train2["YearBuilt"],train2["SalePrice"])
plt.scatter(train2["YearBuilt"],c(train2["YearBuilt"]))

Residual plot

In [ ]:
plt.scatter(train2["YearBuilt"],-(c(train2["YearBuilt"])-train2["SalePrice"]))

In [ ]:
p1=np.polyfit(train2["GrLivArea"],train2["SalePrice"],3)
c1=np.poly1d(p1)
plt.scatter(train2["GrLivArea"],train2["SalePrice"])
plt.scatter(train2["GrLivArea"],c1(train2["GrLivArea"]))

In [ ]:
plt.scatter(train2["GrLivArea"],-(c1(train2["GrLivArea"])-train2["SalePrice"]))

In [ ]:
p=np.polyfit(train2["TotalBsmtSF"][(train2["TotalBsmtSF"]!=0) & (train2["TotalBsmtSF"]<=3000)],train2["SalePrice"][(train2["TotalBsmtSF"]!=0) & (train2["TotalBsmtSF"]<=3000)],3)
c=np.poly1d(p)
plt.scatter(train2["TotalBsmtSF"][train2["TotalBsmtSF"]!=0],train2["SalePrice"][train2["TotalBsmtSF"]!=0])
plt.scatter(train2["TotalBsmtSF"][(train2["TotalBsmtSF"]!=0) & (train2["TotalBsmtSF"]<=3000)],c(train2["TotalBsmtSF"][(train2["TotalBsmtSF"]!=0) & (train2["TotalBsmtSF"]<=3000)]))

Analysing Different Categorical Variables for their relationship with Sale Price

Frow below it can be certian Sub Classes have lower Sale Price specifically 1945 and older ones

In [ ]:
sns.boxplot(x=train2["MSSubClass"],y=train2["SalePrice"])

In [ ]:
sns.boxplot(x=train2["LotShape"],y=train2["SalePrice"])

In [ ]:
sns.boxplot(x=train2["LotConfig"],y=train2["SalePrice"])

In [ ]:
plt.figure(figsize=(20,10))
sns.boxplot(x=train2["Neighborhood"],y=train2["SalePrice"])

In [ ]:
sns.boxplot(x=train2["Condition1"],y=train2["SalePrice"])

In [ ]:
sns.boxplot(x=train2["HouseStyle"],y=train2["SalePrice"])

An important and a strong one relation is seen here which also confirms are HYPOTHESIS that Sale Price is affected by the Overall Qualityof the House 

In [ ]:
sns.boxplot(x=train2["OverallQual"],y=train2["SalePrice"])

In [ ]:
sns.boxplot(x=train2["OverallCond"],y=train2["SalePrice"])

Making Different Linear MOdels on Numerical Fields of the Dataframe

In [ ]:
train3=train2.select_dtypes(exclude="object")
var=train3.columns.tolist()
var.remove("SalePrice")
var.remove("Id")
train3=train3.dropna()

In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(train3[var],train3["SalePrice"],random_state=0)

In [ ]:
xtrain.shape

In [ ]:
from sklearn.linear_model import LinearRegression
lr1=LinearRegression().fit(xtrain,ytrain)
lr1.score(xtrain,ytrain)

In [ ]:
lr1.score(xtest,ytest)

In [ ]:
from sklearn.linear_model import Ridge
rid=Ridge(alpha=120).fit(xtrain,ytrain)
rid.score(xtrain,ytrain)

In [ ]:
rid.score(xtest,ytest)

In [ ]:
from sklearn.linear_model import Lasso
las=Lasso(alpha=120).fit(xtrain,ytrain)
las.score(xtrain,ytrain)

In [ ]:
las.score(xtest,ytest)

In [ ]:
las.coef_

Lasso Model shows highest Accuracy

In [ ]:
test=pd.read_csv("../input/test.csv")

In [ ]:
test1=test.select_dtypes(exclude="object")
var=test1.columns.tolist()
var.remove("Id")

In [ ]:
test2=test1[var].dropna()

Predicted test values 

In [ ]:
testpred=las.predict(test2)

In [ ]:
plt.hist(testpred)